In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [ ]:
import tensorflow as tf
print(tf.__version__) 

2.6.0


In [ ]:
!pwd

/content


In [ ]:
!kaggle competitions download -c digit-recognizer

  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 86.5MB/s]
 55% 5.00M/9.16M [00:00<00:00, 44.2MB/s]
100% 9.16M/9.16M [00:00<00:00, 58.0MB/s]
 82% 5.00M/6.09M [00:00<00:00, 31.4MB/s]
100% 6.09M/6.09M [00:00<00:00, 29.9MB/s]


In [ ]:
!unzip 'train.csv.zip'
!unzip 'test.csv.zip'

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [ ]:
!rm -rf train.csv.zip
!rm -rf test.csv.zip

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [ ]:
train_y=train['label'].astype('float32')
train_x=train.drop(['label'],axis=1).astype('int32')
test_x=test.astype('float32')

In [ ]:
train_x=train_x.values.reshape(-1,28,28,1)
train_x=train_x /255.0
test_x=test_x.values.reshape(-1,28,28,1)
test_x=test_x /255.0

In [ ]:
train_y=tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
  tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='Same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='Same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256,activation='relu'),
  tf.keras.layers.Dense(256,activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10,activation='softmax'),
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6, 6, 64)         

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs=[]):
    if (logs.get('accuracy')> 0.999):
      print('tooop')
      self.model.stop_training =True
callbacks=myCallback()

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0005,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    name='AZdam'
)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=50,epochs=20,callbacks=[callbacks])


Epoch 1/20
840/840 [==============================] - 130s 154ms/step - loss: 0.3395 - accuracy: 0.8872
Epoch 2/20
840/840 [==============================] - 129s 154ms/step - loss: 0.0843 - accuracy: 0.9755
Epoch 3/20
840/840 [==============================] - 129s 154ms/step - loss: 0.0623 - accuracy: 0.9824
Epoch 4/20
840/840 [==============================] - 129s 154ms/step - loss: 0.0499 - accuracy: 0.9852
Epoch 5/20
840/840 [==============================] - 129s 154ms/step - loss: 0.0425 - accuracy: 0.9876
Epoch 6/20
840/840 [==============================] - 130s 155ms/step - loss: 0.0340 - accuracy: 0.9895
Epoch 7/20
840/840 [==============================] - 130s 154ms/step - loss: 0.0315 - accuracy: 0.9907
Epoch 8/20
840/840 [==============================] - 130s 154ms/step - loss: 0.0292 - accuracy: 0.9912
Epoch 9/20
840/840 [==============================] - 129s 153ms/step - loss: 0.0255 - accuracy: 0.9928
Epoch 10/20
840/840 [==============================] - 131s 156m

In [ ]:
results =model.predict(test_x)
results =np.argmax(results,axis=1)
results = pd.Series(results,name='Label')

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name="ImageId"),results],axis=1)
submission.to_csv("submission",index=False)

In [ ]:
!kaggle competitions submit -c digit-recognizer -f submission -m "Message"

100% 208k/208k [00:03<00:00, 65.0kB/s]
Successfully submitted to Digit Recognizer